![Logo UOC](https://www.uoc.edu/portal/_resources/common/imatges/marca_UOC/logotips/logo-UOC-2linies.png)

# TFG - Inteligencia Artificial
Enero de 2026

## Predicción de Respuesta a Tratamientos Oncológicos Basada en el Perfil Genético Mediante Técnicas de Aprendizaje Automático (ML) e Identificación de Genes Candidatos a Biomarcador Usando Técnicas de Explicabilidad (XAI) y Cuantificación de la Incertidumbre (UQ)

---

#### Pablo Vázquez Rodríguez
##### Grado en Ingeniería Informática
##### Inteligencia Artificial

#### Dra. María Moreno de Castro
#### Dr. Friman Sánchez


### Composición del dataset inicial
Desde la web CTR-DB se descargan los datos de cada estudio por separado y aquí simplemente se unen para crear un dataset único sobre el que trabajar.

In [1]:
# Importaciones
import pandas as pd
from pathlib import Path
from git import Repo  # Para clonar el repositorio con los datos no unificados

In [2]:
# Configuración
storage_repo_url = "https://github.com/pvazquezr/TFG_AI_STORAGE"
local_tmp_folder = Path(f"/tmp/TFG_AI_STORAGE")
unstructured_data_folder_name = "CTR_Microarray_Complete_Unstructured_dataset"
unstructured_data_folder_path = Path(f"{local_tmp_folder}/{unstructured_data_folder_name}")

In [3]:
# Función auxiliar para cargar y procesar cada matriz de expresión
def load_expression_matrix(path):
    df = pd.read_csv(path, index_col=0, decimal=',')
    return df

# Función auxiliar para cargar sólo los metadatos necesarios por
# cada estudio
def load_metadata(path):
    meta = pd.read_csv(path)
    return meta[['Sample_id', 'Response']]

In [4]:
# Clonación del repositorio de almacenamiento de los datos no unificados
if local_tmp_folder.exists():
    print("Actualizando repositorio...", end="")
    repo = Repo(local_tmp_folder)
    repo.remotes.origin.pull()
    print(" Hecho!")
else:
    print("Clonando repositorio...", end="")
    repo = Repo.clone_from(storage_repo_url, local_tmp_folder)
    print(" Hecho")

Clonando repositorio... Hecho


In [ ]:
# Lista para almacenar todas las matrices formateadas
formatted_matrix_list = []

# Formatear todas las matrices para incluir respuesta y transponerlas
for folder in unstructured_data_folder_path.iterdir():
    print(f"Procesando {folder} ...", end="")

    # Leer matriz de expresión y metadatos
    expression_matrix = load_expression_matrix(f"{folder}/matrix_.csv")
    metadata = load_metadata(f"{folder}/cli.inf_.csv")
    
    # Transponer para que cada fila corresponda a una muestra
    expression_matrix = expression_matrix.T
    expression_matrix['Sample_id'] = expression_matrix.index
    
    # Añadir la respuesta del tratamiento según corresponde
    formatted_expression_matrix = pd.merge(expression_matrix, metadata, on='Sample_id')

    # Guardar la matriz formateada en la lista
    formatted_matrix_list.append(formatted_expression_matrix)

    print(" Hecho!")

Procesando /tmp/TFG_AI_STORAGE/CTR_Microarray_Complete_Unstructured_dataset/CTR_Microarray_1-I ... Hecho!
Procesando /tmp/TFG_AI_STORAGE/CTR_Microarray_Complete_Unstructured_dataset/CTR_Microarray_10-I ... Hecho!
Procesando /tmp/TFG_AI_STORAGE/CTR_Microarray_Complete_Unstructured_dataset/CTR_Microarray_100-I ... Hecho!
Procesando /tmp/TFG_AI_STORAGE/CTR_Microarray_Complete_Unstructured_dataset/CTR_Microarray_102-I ... Hecho!
Procesando /tmp/TFG_AI_STORAGE/CTR_Microarray_Complete_Unstructured_dataset/CTR_Microarray_104-I ... Hecho!
Procesando /tmp/TFG_AI_STORAGE/CTR_Microarray_Complete_Unstructured_dataset/CTR_Microarray_106-I ... Hecho!
Procesando /tmp/TFG_AI_STORAGE/CTR_Microarray_Complete_Unstructured_dataset/CTR_Microarray_107-I ... Hecho!
Procesando /tmp/TFG_AI_STORAGE/CTR_Microarray_Complete_Unstructured_dataset/CTR_Microarray_120-I ... Hecho!
Procesando /tmp/TFG_AI_STORAGE/CTR_Microarray_Complete_Unstructured_dataset/CTR_Microarray_129-I ... Hecho!
Procesando /tmp/TFG_AI_STORAGE/

In [ ]:
# Unir todos los datasets
expression_df = pd.concat(formatted_matrix_list, ignore_index=True)
print(expression_df.shape)

In [ ]:
# Eliminar todos los genes que no tengan datos de expresión
# para todas las muestras

# Hacemos primero un double check para asegurarnos de que
# no se haya colado ningún campo vacío en Sample_id ni Response
empty_sample_id = expression_df["Sample_id"].isna().any()
empty_response_id = expression_df["Response"].isna().any()

if empty_sample_id or empty_response_id:
    raise Exception("HAY DATOS VACÍOS EN SAMPLE_ID O RESPONSE. PROCESO ABORTADO")
else:
    # Ahora con seguridad podemos eliminar los genes (columnas) con
    # algún dato vacío
    final_expression_df = expression_df.dropna(axis=1, how="any")

In [ ]:
print(final_expression_df.shape)
print(final_expression_df.head())

Tras rematar el formateo y composición del dataset:
* Teníamos: 1889 muestras asociadas con 26011 genes con su expresión génica
* Acabamos con: 1889 muestras asociadas con 2396 genes con su expresión génica

Aunque la reducción de genes es muy significativa, para nuestro estudio necesitamos priorizar el número de muestras para poder entrenar a nuestro modelo, aunque ello implique tener que reducir el número de genes por no disponer de datos de expresión para todas las muestras.

In [ ]:
# Guardamos el dataset ya formateado para procesar
final_expression_df.to_csv("./data/CTR_Fluorouracil_expresion_and_response_dataset.csv")